In [ ]:
def generate_descriptors(file_path, smiles_column_name, phase_column_name, status_column_name, disease_column_name):

        data = pd.read_csv(file_path)[[smiles_column_name, phase_column_name, status_column_name, disease_column_name]]

        unique_data = data.dropna(subset=[smiles_column_name, disease_column_name]).drop_duplicates(subset=[smiles_column_name, disease_column_name]).reset_index(drop=True)

        invalid_smiles = []
        for smile in  unique_data[smiles_column_name]:
          mol = Chem.MolFromSmiles(smile)
          if mol is None:
           invalid_smiles.append(smile)

        verify_data =  unique_data[~ unique_data[smiles_column_name].isin(invalid_smiles)]

        desc_list = Chem.Descriptors.descList
        unique_desc_set = set(desc_list)
        unique_desc_list = list(unique_desc_set)

        descriptors = [x[0] for x in unique_desc_list]
        calculator = MolecularDescriptorCalculator(descriptors)

        dataset_with_descriptors = []

        for smile in verify_data[smiles_column_name]:
         mol = Chem.MolFromSmiles(smile)
         descriptors_values = calculator.CalcDescriptors(mol)
         dataset_with_descriptors.append(descriptors_values)
        dataset_with_descriptors = pd.DataFrame(dataset_with_descriptors, columns=descriptors)

        descripted_data = pd.concat([verify_data.reset_index(drop=True), dataset_with_descriptors], axis=1)

        return descripted_data